In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PyEMD import CEEMDAN
from copy import deepcopy
from scipy.fft import fft
import pywt

In [ ]:
# Generate synthetic stress-strain data with noise
# 1. Generate noise based on sine function
def noise_generater(x_sammpling, freq, amp, decay):
    return amp * np.sin(2 * np.pi * freq * x_sammpling) * decay

# Generate synthetic stress-strain data
def generate_stress_strain_data(N, noise=True):
    # Base data of stress-strain curve with non-linear trend
    x = np.linspace(0, 1, N)
    y_trend = 300 * x ** 0.5  # 非線形な応力ひずみ曲線

    # Frequency and amplitude settings for noise
    freq_1 = 5
    freq_2 = 50
    # freq_3 = 120  # Frequency for the third noise component

    amp_1 = -40
    amp_2 = 1
    # amp_3 = 2

    decay_1 = np.exp(-0.5 * x)
    decay_2 = np.exp(-0.001 * x)
    # decay_3 = np.exp(-1 * x)

    noise_1 = noise_generater(x, freq_1, amp_1, decay_1)
    noise_high_freq = deepcopy(noise_1)
    noise_2 = noise_generater(x, freq_2, amp_2, decay_2)
    noise_high_freq += noise_2

    # noise_3 = noise_generater(x, freq_3, amp_3, decay_3)
    # noise_high_freq += noise_3

    # noise_4 = noise_generater(x, freq_3, amp_3, decay_3)
    # noise_high_freq += noise_4

    # White noise
    noise_random = 0.1 * np.random.randn(N)

    # Combine all noise components
    if noise:
        y = y_trend + noise_high_freq + noise_random
    else:
        y = y_trend

    # return x, y, [noise_1, noise_2, noise_3, noise_random]
    return x, y, [noise_1, noise_2, noise_random]

def mirror_point_symmetric(x, y):
    # point symmetric extension
    x_mirror = -x[::-1]  # xを反転（時間軸を反転）
    y_mirror = -y[::-1]  # yも符号を反転
    x_extended = np.concatenate([x_mirror, x])
    y_extended = np.concatenate([y_mirror, y])
    return x_extended, y_extended




In [ ]:
def is_periodic(imf, threshold=0.1):
    spectrum = np.abs(fft(imf))
    spectrum = spectrum[:len(spectrum)//2]  # 正の周波数のみ
    dominant_ratio = np.max(spectrum[1:]) / np.sum(spectrum[1:] + 1e-8)  # DC除去
    return dominant_ratio > threshold

# IMFを選別し、周期的な成分を除去
def imf_fft_filter(IMFs, threshold=0.1, verbose=False):
    selected_imfs = []
    for i, imf in enumerate(IMFs):
        periodic = is_periodic(imf, threshold=threshold)
        if verbose:
            status = "Removed" if periodic else "Kept"
            print(f"IMF {i}: {status}")
        if not periodic:
            selected_imfs.append(imf)
    return np.sum(selected_imfs, axis=0)

# Waveletフィルタ
def wavelet_denoise(signal, wavelet='db4', level=3):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    coeffs[1:] = [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)[:len(signal)]

# main
N = 500
x, y_noisy, noises = generate_stress_strain_data(N, noise=True)
_, y_true, _ = generate_stress_strain_data(N, noise=False)

x_ext, y_noisy_ext = mirror_point_symmetric(x, y_noisy)

# IMF Decomposition by CEEMDAN
ceemdan = CEEMDAN()
ceemdan.trials = 10000
ceemdan.noise_seed = 42
IMFs = ceemdan(y_noisy_ext)



In [ ]:

# Smoothing using demaped sine model. For this high strain-rate problem, the shock wave is assume to be a major factor of the oscillation,
# and can be modeled as a damped sine wave.
from scipy.optimize import curve_fit
def damped_sine(t, A, alpha, f, phi):
    return A * np.exp(-alpha * t) * np.sin(2 * np.pi * f * t + phi)

def fit_damped_model(x, imf):
    guess = [np.std(imf), 2.0, 5.0, 0.0]  # Initial values: 振幅、減衰、周波数、位相
    try:
        popt, _ = curve_fit(damped_sine, x, imf, p0=guess, maxfev=5000000)
        print(f"Fitted parameters: {popt}")
        return popt
    except RuntimeError:
        print("Fit failed")
        return [0, 0, 0, 0]



In [ ]:
# 各IMFに対してフィッティング
fitted_imfs = []
params_list = []
num_IMFs = len(IMFs)
plt.figure(figsize=(12, num_IMFs*3))
plt.subplot(num_IMFs + 2, 1, 1)
plt.plot(x, y_noisy, label='Original curve with Noise')
plt.legend()

for i, imf in enumerate(IMFs):
    params = fit_damped_model(x_ext, imf)
    print(f"IMF {i + 1} params: {params}")
    if not np.all(params == [0, 0, 0, 0]):
        fitted_imf = damped_sine(x_ext, *params)
        fitted_imfs.append(fitted_imf)
        params_list.append(params)
    else:
        fitted_imfs.append(np.zeros_like(x_ext))
        params_list.append(params)

    plt.subplot(num_IMFs + 2, 1, i + 2)
    plt.plot(x_ext, imf, label=f'IMF {i + 1}', alpha=0.5)
    plt.plot(x_ext, fitted_imf, label=f'Fitted_Model {i + 1}')

index_from_last_main_component = 1
reconstructed = np.sum(fitted_imfs[-index_from_last_main_component:], axis=0)

# 再構成した信号のプロット
plt.subplot(num_IMFs + 2, 1, num_IMFs + 2)
plt.plot(x_ext, y_noisy_ext, label='Original Curve', alpha=0.5)
plt.plot(x_ext, reconstructed, label='Resolved Result', linewidth=2)
plt.plot(x, y_true, label='True Curve before adding noise', linestyle='--')
plt.plot(x_ext, np.sum(IMFs, axis=0)-np.array(fitted_imfs[2])-np.array(fitted_imfs[3]), label='All - fitted_imfs[2,:]', linewidth=2)

plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(x_ext, y_noisy_ext, label='Original Curve', alpha=0.5)
# plt.plot(x_ext, reconstructed, label='Resolved Result', linewidth=2)
plt.plot(x, y_true, label='True Curve before adding noise', linestyle='--')
# plt.plot(x_ext, np.sum(IMFs, axis=0)-np.array(fitted_imfs[2])-np.array(fitted_imfs[3]), label='All - fitted_imfs[2,:]', linewidth=2)
plt.plot(x_ext, y_noisy_ext -np.array(fitted_imfs[2])-np.array(fitted_imfs[3]), label='All - fitted_imfs[2,:]', linewidth=2)
plt.plot(x_ext, y_noisy_ext -np.array(IMFs[2])-np.array(IMFs[3]), label='All - imfs[2,:]', linewidth=2)


plt.xlim(0, 1)
plt.ylim(0, 320)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
np.sqrt(np.sum((extract_second_half(np.sum(IMFs, axis=0)-np.array(fitted_imfs[2])-np.array(fitted_imfs[3])) - y_true)**2))

In [ ]:
np.sqrt(np.sum(extract_second_half((extract_second_half(np.sum(IMFs, axis=0)-np.array(fitted_imfs[2])-np.array(fitted_imfs[3])) - y_true))**2))

減衰振動モデルのフィッティングをMirroring前のデータのみを用いて実施
期待:
1. 周波数が小さく振幅が大きな波に対して→減衰振動モデルの時間遅れのフィッテイング精度向上
2. 周波数が大きく振幅が小さな波に対して→IMFに分解した際に点対称の点（引張開始直後）が大きな振幅となり分解される傾向があるが、IMFを用いて減衰振動関数をフィッテイングをする際に点対称とすると
減衰する様子のフィッテイングができない

In [ ]:
def extract_second_half(x):
    # xの後半部分を抽出
    mid_index = len(x) // 2
    x_half = x[mid_index:]
    return x_half

In [ ]:
# 各IMFに対してフィッティング
fitted_imfs = []
params_list = []
plt.figure(figsize=(12, num_IMFs*3))
plt.subplot(num_IMFs + 2, 1, 1)
plt.plot(x, y_noisy, label='Original curve with Noise')
plt.legend()

for i, imf in enumerate(IMFs):
    params = fit_damped_model(extract_second_half(x_ext), extract_second_half(imf))
    print(f"IMF {i + 1} params: {params}")
    if not np.all(params == [0, 0, 0, 0]):
        fitted_imf = damped_sine(extract_second_half(x_ext), *params)
        fitted_imfs.append(fitted_imf)
        params_list.append(params)
    else:
        fitted_imfs.append(np.zeros_like(extract_second_half(x_ext)))
        params_list.append(params)

    plt.subplot(num_IMFs + 2, 1, i + 2)
    plt.plot(extract_second_half(x_ext), extract_second_half(imf), label=f'IMF {i + 1}', alpha=0.5)
    plt.plot(extract_second_half(x_ext), fitted_imf, label=f'Fitted_Model {i + 1}')

index_from_last_main_component = 1
reconstructed = np.sum(fitted_imfs[-index_from_last_main_component:], axis=0)

# 再構成した信号のプロット
plt.subplot(num_IMFs + 2, 1, num_IMFs + 2)
plt.plot(extract_second_half(x_ext), extract_second_half(y_noisy_ext), label='Original Curve', alpha=0.5)
plt.plot(extract_second_half(x_ext), reconstructed, label='Resolved Result', linewidth=2)
plt.plot(x, y_true, label='True Curve before adding noise', linestyle='--')
plt.plot(extract_second_half(x_ext), extract_second_half(np.sum(IMFs, axis=0))-np.array(fitted_imfs[2])-np.array(fitted_imfs[3]), label='All - fitted_imfs[2,:]', linewidth=2)

plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
np.sqrt(np.sum((extract_second_half(np.sum(IMFs, axis=0))-np.array(fitted_imfs[2])-np.array(fitted_imfs[3]) - y_true)**2))

In [ ]:
np.sqrt(np.sum(extract_second_half(extract_second_half(np.sum(IMFs, axis=0))-np.array(fitted_imfs[2])-np.array(fitted_imfs[3]) - y_true)**2))